In [1]:
import json
import pandas as pd
import os

import PySAM
import PySAM.Pvsamv1 as pv_model
import PySAM.Grid as grid
import PySAM.Utilityrate5 as utility_rate
import PySAM.ResourceTools as tools

from functools import partial
import itertools
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from operator import sub
import seaborn as sns

In [2]:
def get_cmod_json(json_file_path):
    with open(json_file_path, 'r') as f:
        dict = json.load(f)
    return dict

In [3]:
locs = pd.read_excel("Coordinates for Representative Microgrid Towns.xlsx", header=0)

kw_loads = pd.read_excel("Sample Agvoltaic PUE Load Profile.xlsx", header=[0, 1], sheet_name="100 kW Load Profiles")

mw_loads = pd.read_excel("Sample Agvoltaic PUE Load Profile.xlsx", header=[0, 1], sheet_name="1 MW Load Profiles")

configs = ["Elevated-1MW", "Elevated-100kW", "Elevated-Checkerboard-1MW", "Elevated-Checkerboard-100kW", \
           "Standard-1MW", "Standard-100kW", "Wider-Row-1MW", "Wider-Row-100kW"]

loads = ["CROP Archetype", "LIVESTOCK Archetype - Chickens", "LIVESTOCK Archetype - Sheep / Cattle"]

In [5]:
print(kw_loads)

     Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                   Date              Month               Hour   
0            2023-01-01            January                  0   
1                   NaT            January                  1   
2                   NaT            January                  2   
3                   NaT            January                  3   
4                   NaT            January                  4   
...                 ...                ...                ...   
8755                NaT           December               8755   
8756                NaT           December               8756   
8757                NaT           December               8757   
8758                NaT           December               8758   
8759                NaT           December               8759   

        CROP Archetype                                    \
     Cold Storage (kW)  PUE (kW) Cold Storage + PUE (kW)   
0                 0.54  0.089716  

In [4]:
data_folder = "SAM_files/"
output_folder = "results/"

load_archetypes = {
    "CROP Archetype" : "Cold Storage + PUE (kW)", 
    "LIVESTOCK Archetype - Chickens" : "Cold Storage + PUE (kW)",
    "LIVESTOCK Archetype - Sheep / Cattle" : "Cold Storage (kW)"
}

data_cols = ["Energy Produced (kWh)", "Monthly Load (kWh)", "Energy for other uses (kWh)"]

data_cols_daily = ["Hourly capacity min (kW)", "Hourly capacity avg (kW)", "Hourly capacity max (kW)"]

#value = "100" in c


In [5]:
multi_df_monthly = {}
multi_df_hourly = {}

"""
config = "Standard-100kW"
idx = 0

load_type = "CROP Archetype"
"""
for idx in range(0, 10):
    for config in configs:
        for load_type in load_archetypes:
            pv = pv_model.default("PVBatteryCommercial")

            pv_json = get_cmod_json(data_folder + config + "_pvsamv1.json")
            grid_json = get_cmod_json(data_folder + config + "_grid.json")
            ur_json = get_cmod_json(data_folder + config + "_utilityrate5.json")

            lat = "%.8f" % float(locs.loc[idx]["Lat"])
            long = "%.8f" % float(locs.loc[idx]["Long"])
            name = locs.loc[idx]["Departement"]

            print(lat, long, name, config, load_type)

            for k, v in pv_json.items():
                if "adjust_" in k:
                    k = k.replace("adjust_", "")
                try:
                    pv.value(k, v)
                except AttributeError:
                    print("Error ", k)

            pv.SolarResource.solar_resource_file = "weather_files/" + str(lat) + "_" + str(long) +"_psm3-tmy_60_tmy.csv"
            sub_load = load_archetypes[load_type]

            if "100" in config:
                pv.Load.load = kw_loads[load_type, sub_load].values
            else:
                pv.Load.load = mw_loads[load_type, sub_load].values

            gr = grid.from_existing(pv)

            for k, v in grid_json.items():
                try:
                    gr.value(k, v)
                except AttributeError:
                    print("Error ", k)

            ur = utility_rate.from_existing(pv)

            for k, v in ur_json.items():
                try:
                    ur.value(k, v)
                except AttributeError:
                    print("Error ", k)

            pv.execute()
            gr.execute()
            ur.execute()

            monthly_energy = pv.Outputs.monthly_energy
            monthly_load = ur.Outputs.year1_monthly_load
            monthly_to_grid = list (map(sub, monthly_energy, monthly_load)) # Includes inverter losses. utility rate monthly output seems broken right now

            crop_shading = pv.Outputs.subarray1_ground_rear_spatial

            df = pd.DataFrame([monthly_energy, monthly_load, monthly_to_grid], index=data_cols, columns=range(1,13))

            full_name = name + " - " + config + " - " + load_type.replace("/", "-")

            multi_df_monthly[full_name] = df

            batt_cap = pv.BatterySystem.batt_power_discharge_max_kwac

            hourly_pv_gen = pv.Outputs.gen_without_battery
            hourly_batt_to_load = pv.Outputs.batt_to_load
            hourly_pv_to_load = pv.Outputs.system_to_load
            hourly_batt_charging = pv.Outputs.system_to_batt

            hourly_min = [1e7] * 24
            hourly_avg = [0] * 24
            hourly_max = [0] * 24

            for i in range(0, 8760):
                hourly_capacity = batt_cap - pv.Load.load[i] + hourly_pv_gen[i] - hourly_batt_charging[i]
                j = i % 24

                hourly_min[j] = min(hourly_min[j], hourly_capacity)
                hourly_max[j] = max(hourly_max[j], hourly_capacity)
                hourly_avg[j] = hourly_avg[j] + hourly_capacity

            for i in range(0, 24):
                hourly_avg[i] = hourly_avg[i] / 365

            df_daily = pd.DataFrame([hourly_min, hourly_avg, hourly_max], index=data_cols_daily, columns=range(0,24))
            multi_df_hourly[full_name] = df_daily


        # Don't run this for every load (once per weatherfile/config pair)
        cs_list = list(crop_shading)
        df_crops = pd.DataFrame(cs_list[1:8761], columns=cs_list[0])
        df_crops = df_crops.rename(columns={0: 'Timestep'})
        df_crops.set_index('Timestep', inplace=True)
        df_crops = df_crops.assign(ghi=pv.Outputs.gh_calc[0:8760])
        cs_name = name + " - " + config 
        df_crops.to_csv(output_folder + cs_name + "_crop_shading.csv")

final_df = pd.concat(multi_df_monthly, axis=0)

final_df.to_csv("Haiti_energy_outputs.csv")

hourly_df = pd.concat(multi_df_hourly, axis=0)

hourly_df.to_csv("Haiti_power_outputs.csv")

18.27459067 -74.15511722 Sud Elevated-1MW CROP Archetype
18.27459067 -74.15511722 Sud Elevated-1MW LIVESTOCK Archetype - Chickens
18.27459067 -74.15511722 Sud Elevated-1MW LIVESTOCK Archetype - Sheep / Cattle
18.27459067 -74.15511722 Sud Elevated-100kW CROP Archetype
18.27459067 -74.15511722 Sud Elevated-100kW LIVESTOCK Archetype - Chickens
18.27459067 -74.15511722 Sud Elevated-100kW LIVESTOCK Archetype - Sheep / Cattle
18.27459067 -74.15511722 Sud Elevated-Checkerboard-1MW CROP Archetype
18.27459067 -74.15511722 Sud Elevated-Checkerboard-1MW LIVESTOCK Archetype - Chickens
18.27459067 -74.15511722 Sud Elevated-Checkerboard-1MW LIVESTOCK Archetype - Sheep / Cattle
18.27459067 -74.15511722 Sud Elevated-Checkerboard-100kW CROP Archetype
18.27459067 -74.15511722 Sud Elevated-Checkerboard-100kW LIVESTOCK Archetype - Chickens
18.27459067 -74.15511722 Sud Elevated-Checkerboard-100kW LIVESTOCK Archetype - Sheep / Cattle
18.27459067 -74.15511722 Sud Standard-1MW CROP Archetype
18.27459067 -74.1

In [22]:
print(df_daily)

                                 0          1          2          3   \
Hourly capacity min (kW)  33.814710  33.814710  33.814710  33.814710   
Hourly capacity avg (kW)  33.827692  33.827692  33.827692  33.827692   
Hourly capacity max (kW)  33.839994  33.839994  33.839994  33.839994   

                                 4          5          6          7   \
Hourly capacity min (kW)  33.814710  33.814710  33.814710  32.981572   
Hourly capacity avg (kW)  33.827692  33.829820  38.598697  56.728304   
Hourly capacity max (kW)  33.839994  34.073638  43.881781  67.626127   

                                 8           9   ...          14         15  \
Hourly capacity min (kW)  34.671144   42.556519  ...   32.764154  32.602317   
Hourly capacity avg (kW)  77.104623   89.626483  ...   75.752538  64.384169   
Hourly capacity max (kW)  88.007397  103.456360  ...  102.579993  86.349438   

                                 16         17         18         19  \
Hourly capacity min (kW)  32.3197

In [32]:
final_df.index.isin(('100kW', any()))

TypeError: any() takes exactly one argument (0 given)